In [1]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import pysal.model.spreg as spreg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal.lib
import geopandas as gpd
import pandas as pd
from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
import pandas as pd
from scipy.stats import pearsonr
from esda.moran import Moran
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import spreg
from spreg import SURlagIV
from spreg import GM_Lag
import libpysal
from spreg import AKtest
from spreg import ML_Lag
from spreg import GM_Combo

**Loading Path**

In [2]:
# Dependent variable
dep_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

# Independent variable
ind_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\1.2 Saving the Independent Variables"

# Steamengines and horsepower
power_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

In [3]:
# lets read the metadata
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\control.dta") as reader:
    meta_control = reader.variable_labels()
    
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\_FINAL_c_steamenginges_1904.dta") as reader:
    meta_steam = reader.variable_labels()

In [4]:
meta_control

{'id_inequality': 'County ID',
 'rb_id': 'District ID',
 'rb_name': 'District name',
 'province_id': 'Province ID',
 's_industry': 'Share industry employment',
 's_prot': 'Share Protestants',
 's_german': 'Share German-speaking',
 'longitud': 'Longitude',
 'latitud': 'Latitude',
 'slope_km': 'Terrain slope',
 'soil_suitability': 'Soil suitability',
 'mean_temp': 'Average temperature',
 'mean_preci': 'Average precipitation',
 'dist_100km': 'Distance to the nearest river',
 'log_distance_carbon_c': 'Least distance to carbon field (in log)'}

In [5]:
# Load Skill levels per Landkreis
skills_df = pd.read_csv(f"{dep_var_path}\Main_Dep_WFH.csv",
                        sep=",")

# Load control variables
control_df = pd.read_csv(f"{ind_var_path}\Independent Variables.csv",
                         sep=",")

# Load SteamEngine and Horsepower data
power_df = pd.read_stata(f"{power_path}\_FINAL_c_steamenginges_1904.dta")

In [6]:
# Merge the dataframes using an inner join on the "id_inequality" column
prep_df = pd.merge(control_df, skills_df, on="id_inequality", how="inner")
merged_df = pd.merge(prep_df, power_df, on="id_inequality", how="inner")

In [7]:
# Transform steamengines to steamengines per worker
merged_df["steamengines_number"] = (merged_df["steamengines_number"] / merged_df["d_Hauptberuf"])*1000
merged_df["horsepower"] = (merged_df["horsepower"] / merged_df["d_Hauptberuf"])*1000

merged_df["steamengines_number"] = np.log(merged_df["steamengines_number"])
merged_df["horsepower"] = np.log(merged_df["horsepower"])

In [8]:
merged_df["per_unskilled"] = merged_df["unskilled"] / merged_df["d_Hauptberuf"]
merged_df["per_low"] = merged_df["low"] / merged_df["d_Hauptberuf"]
merged_df["per_medium"] = merged_df["medium"] / merged_df["d_Hauptberuf"]
merged_df["per_high"] = merged_df["high"] / merged_df["d_Hauptberuf"]

merged_df["per_unskilled"] = np.log(merged_df["per_unskilled"])
merged_df["per_low"] = np.log(merged_df["per_low"])
merged_df["per_medium"] = np.log(merged_df["per_medium"])
merged_df["per_high"] = np.log(merged_df["per_high"])

In [9]:
# Drop Erblanden
#district_Erblanden = [31]
#merged_df = merged_df[~merged_df["rb_id"].isin(district_Erblanden)]

# Make Dummies for FE

In [10]:
dummies = pd.get_dummies(merged_df["province_id"], drop_first=True) * 1
merged_df = pd.concat([merged_df, dummies], axis=1)

# Convert column names to strings
merged_df.columns = [str(col) for col in merged_df.columns]

In [11]:
# Dictionary
number_words = {
    "1": "one", "2": "two", "3": "three", "4": "four", "5": "five",
    "6": "six", "7": "seven", "8": "eight", "9": "nine", "10": "ten",
    "11": "eleven", "12": "twelve", "13": "thirteen", "14": "fourteen"
}

# Transform the column names
merged_df.columns = [number_words[col.split('.')[0]] if col.split('.')[0] in number_words else col for col in merged_df.columns]

merged_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,five,six,seven,eight,nine,ten,eleven,twelve,thirteen,fourteen
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,0,0,0,0,0,0,0,0,0,0
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,0,0,0,0,0,0,0,0,0,0
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,0,0,0,0,0,0,0,0,0,0
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,0,0,0,0,0,0,0,0,0,0
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,0,0,0,0,0,0,0,1,0,0
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,0,0,0,0,0,0,0,0,0,1
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,0,0,0,0,0,0,0,0,0,1
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,0,0,0,0,0,0,0,0,0,1


# 2SLS
Lets run a good'ol reliable IV first

## 2SLS function
I build a function that runs multiple 2SLS in a row, each time adding another variable from a list of control variables

In [12]:
def twoSLS(df, IV, first_stage_dependent, controls, second_stage_independent):
    """
    Performs a series of Two-Stage Least Squares (2SLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    second_stage_independent (str): The name of the main independent variable for the second stage regression.

    Returns:
    tuple of DataFrames: The first DataFrame contains the results (coefficients, significance, 
                         Adj. R-squared, F-statistic) of each first stage regression. 
                         The second DataFrame contains the results 
                         of each second stage regression. Significance is denoted by stars: 
                         *** for p < 0.01, ** for p < 0.05, and * for p < 0.1.
                         The third DataFrame contains the residuals.
    """

    first_stage_results = []
    second_stage_results = []
    # For analysis of spatial autocorrelation
    residuals = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = " + ".join(subset_controls)
        if controls_formula_part:
            controls_formula_part = " + " + controls_formula_part

        # To avoid SettingWithCopyWarning
        df_copy = df.copy()

        # First Stage Regression
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()

        # Add the predicted values as a new column in the copy
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression
        second_stage_formula = f"{second_stage_independent} ~ predicted{controls_formula_part}"
        second_stage_model = smf.ols(second_stage_formula, data=df_copy)
        second_stage_fit = second_stage_model.fit()

        def format_coef(coef, std_err, p_value):
            stars = ""
            if p_value < 0.01:
                stars = "***"
            elif p_value < 0.05:
                stars = "**"
            elif p_value < 0.1:
                stars = "*"
    
            # Check if coef or std_err is smaller than 0.000 and format accordingly
            coef_str = f"{coef:.10e}" if abs(coef) < 0.000 else f"{coef:.3f}"
            std_err_str = f"{std_err:.10e}" if abs(std_err) < 0.000 else f"{std_err:.3f}"

            return f"{coef_str}{stars} ({std_err_str})"


        # Extract coefficients and significance of first stage
        first_stage_summary = first_stage_fit.summary2().tables[1]
        first_stage_coefs = first_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Extract coefficients and significance of second stage
        second_stage_summary = second_stage_fit.summary2().tables[1]
        second_stage_coefs = second_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Adding R-squared and F-statistic
        first_stage_coefs.loc["Adj. R-squared"] = f"{first_stage_fit.rsquared_adj:.3f}"
        first_stage_coefs.loc["F-statistic"] = f"{first_stage_fit.fvalue:.3f}"
        second_stage_coefs.loc["Adj. R-squared"] = f"{second_stage_fit.rsquared_adj:.3f}"
        
        # Adding Standard Errors
        second_stage_se = second_stage_fit.bse["predicted"]
        second_stage_coefs.loc["std err"] = second_stage_se
        
        # Residuals
        res = second_stage_fit.resid
        
        # Append 
        first_stage_results.append(first_stage_coefs)
        second_stage_results.append(second_stage_coefs)
        residuals.append(res)

    # Convert the results to DataFrame
    first_stage_df = pd.DataFrame(first_stage_results).T
    second_stage_df = pd.DataFrame(second_stage_results).T
    
    # Df of residuals
    residuals_df = pd.DataFrame(residuals).T

    return first_stage_df, second_stage_df, residuals_df

## Use the Function

In [13]:
# DataFrame that contains all the variables for the regression analysis
dep_var = merged_df[["log_distance_carbon_c", 
                     "s_industry",
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km",
                     "uni_dummy", 
                     "min_distance_uni", 
                     "prov_cap_dummy", 
                     "BER_dummy", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high",
                     "d_Hauptberuf",
                     "horsepower",
                     "two", 
                     "three", 
                     "four", 
                     "five", 
                     "six", 
                     "seven", 
                     "eight", 
                     "nine", 
                     "ten", 
                     "eleven", 
                     "twelve", 
                     "thirteen", 
                     "fourteen"]]

In [14]:
# Make two lists of exogenous variables
# Exclude latitude and longitude once
control_list = ["longitud",
                "latitud",
                "soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "two", 
                "three", 
                "four", 
                "five", 
                "six", 
                "seven", 
                "eight", 
                "nine", 
                "ten", 
                "eleven", 
                "twelve", 
                "thirteen", 
                "fourteen"]

control_list_2 = ["soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "two", 
                "three", 
                "four", 
                "five", 
                "six", 
                "seven", 
                "eight", 
                "nine", 
                "ten", 
                "eleven", 
                "twelve", 
                "thirteen", 
                "fourteen"]

In [15]:
first_unskilled, second_unskilled, res_unskilled = twoSLS(dep_var,
                                                          "log_distance_carbon_c",
                                                          "steamengines_number",
                                                          control_list,
                                                          "per_unskilled")

first_low, second_low, res_low = twoSLS(dep_var,
                                        "log_distance_carbon_c",
                                        "steamengines_number",
                                        control_list,
                                        "per_low")

first_medium, second_medium, res_medium = twoSLS(dep_var,
                                                 "log_distance_carbon_c",
                                                 "steamengines_number",
                                                 control_list,
                                                 "per_medium")

first_high, second_high, res_high = twoSLS(dep_var,
                                           "log_distance_carbon_c",
                                           "steamengines_number",
                                           control_list,
                                           "per_high")

In [16]:
first_unskilled[[1,2,3,24]]

,1,2,3,24
Intercept,2.646*** (0.075),-5.261*** (1.133),-5.615*** (1.110),-6.132** (2.391)
log_distance_carbon_c,-0.095*** (0.016),-0.188*** (0.021),-0.191*** (0.020),-0.169*** (0.023)
Adj. R-squared,0.053,0.126,0.166,0.351
F-statistic,17.174,28.697,29.475,13.914
longitud,0.008 (0.005),0.001 (0.005),-0.004 (0.005),0.098*** (0.024)
latitud,NaN,0.160*** (0.023),0.174*** (0.023),0.081* (0.044)
soil_suitability,NaN,NaN,-0.117*** (0.022),-0.116*** (0.021)
mean_temp,NaN,NaN,NaN,0.331*** (0.043)
mean_preci,NaN,NaN,NaN,0.009*** (0.003)
dist_100km,NaN,NaN,NaN,-0.097 (0.114)


In [17]:
second_unskilled[[1,2,3,24]]

,1,2,3,24
Intercept,0.032 (0.215),-1.690*** (0.470),-1.765*** (0.469),-3.955*** (1.214)
predicted,-0.391*** (0.087),-0.232*** (0.057),-0.226*** (0.056),-0.205*** (0.070)
Adj. R-squared,0.095,0.095,0.094,0.329
std err,0.08658,0.057317,0.056301,0.06988
longitud,0.015*** (0.003),0.012*** (0.003),0.010*** (0.003),-0.004 (0.014)
latitud,NaN,0.026*** (0.009),0.029*** (0.009),0.097*** (0.022)
soil_suitability,NaN,NaN,-0.035*** (0.013),-0.028** (0.014)
mean_temp,NaN,NaN,NaN,-0.111*** (0.034)
mean_preci,NaN,NaN,NaN,-0.004** (0.002)
dist_100km,NaN,NaN,NaN,0.003 (0.059)


In [18]:
second_low[[1,2,3,24]]

,1,2,3,24
Intercept,-3.323*** (0.236),2.493*** (0.514),2.640*** (0.512),4.393*** (1.380)
predicted,0.888*** (0.095),0.351*** (0.063),0.340*** (0.061),0.308*** (0.079)
Adj. R-squared,0.261,0.268,0.272,0.415
std err,0.095267,0.062768,0.061424,0.079449
longitud,-0.026*** (0.003),-0.017*** (0.003),-0.014*** (0.003),-0.029* (0.016)
latitud,NaN,-0.089*** (0.010),-0.095*** (0.010),-0.126*** (0.025)
soil_suitability,NaN,NaN,0.068*** (0.015),0.052*** (0.016)
mean_temp,NaN,NaN,NaN,0.039 (0.038)
mean_preci,NaN,NaN,NaN,0.004* (0.002)
dist_100km,NaN,NaN,NaN,-0.039 (0.067)


In [19]:
second_medium[[1,2,3,24]]

,1,2,3,24
Intercept,-1.265*** (0.157),-2.667*** (0.336),-2.680*** (0.334),-2.012** (0.881)
predicted,-0.002 (0.063),0.128*** (0.041),0.129*** (0.040),0.088* (0.051)
Adj. R-squared,0.044,0.080,0.089,0.299
std err,0.063225,0.041033,0.040107,0.050725
longitud,-0.010*** (0.002),-0.012*** (0.002),-0.012*** (0.002),0.014 (0.010)
latitud,NaN,0.021*** (0.007),0.022*** (0.007),-0.013 (0.016)
soil_suitability,NaN,NaN,-0.006 (0.010),-0.008 (0.010)
mean_temp,NaN,NaN,NaN,0.078*** (0.025)
mean_preci,NaN,NaN,NaN,0.002 (0.001)
dist_100km,NaN,NaN,NaN,0.016 (0.043)


In [20]:
second_high[[1,2,3,24]]

,1,2,3,24
Intercept,-5.558*** (0.396),-5.691*** (0.860),-5.586*** (0.858),-0.398 (2.269)
predicted,0.401** (0.160),0.414*** (0.105),0.406*** (0.103),0.529*** (0.131)
Adj. R-squared,0.051,0.066,0.064,0.277
std err,0.159643,0.104869,0.103054,0.13066
longitud,-0.022*** (0.005),-0.022*** (0.005),-0.020*** (0.005),-0.020 (0.026)
latitud,NaN,0.002 (0.017),-0.002 (0.017),-0.105** (0.040)
soil_suitability,NaN,NaN,0.049** (0.024),0.070*** (0.026)
mean_temp,NaN,NaN,NaN,0.072 (0.063)
mean_preci,NaN,NaN,NaN,0.001 (0.004)
dist_100km,NaN,NaN,NaN,-0.073 (0.110)
